In [14]:
import os
import json
import pandas as pd
import traceback
import requests

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
KEY=os.getenv("GROQ_API_KEY")
MODEL ="llama-3.1-8b-instant"
CHAT_URL = "https://api.groq.com/openai/v1/chat/completions"

In [ ]:
def groq_chat(messages, model=MODEL, temperature=0.1, max_tokens=1024, timeout=60):
    headers = {
        "Authorization": f"Bearer {KEY}",
        "Content-Type": "application/json",
    }
    payload = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
    }
    resp = requests.post(CHAT_URL, headers=headers, json=payload, timeout=timeout)
    if resp.status_code == 429:
        retry_after = resp.headers.get("retry-after")
        raise RuntimeError(f"Rate limited (429). retry-after={retry_after}. Response: {resp.text}")

    resp.raise_for_status()
    data = resp.json()
    content = data.get("choices", [{}])[0].get("message", {}).get("content")
    return content, data, resp.headers

In [34]:
SYSTEM_GEN  = {
    "role": "system",
    "content": (
        "You are an expert MCQ author. Output ONLY valid JSON.\n"
        "Structure: a JSON object where keys are string numbers starting from 1. Each value is an object with:\n"
        "- mcq: string (question)\n"
        "- options: {a,b,c,d}\n"
        "- correct: one of 'a'|'b'|'c'|'d'\n"
        "- explanation: optional one-line explanation\n"
        "Do not add any text outside the JSON object."
    )
}

BASE_USER_TEMPLATE = (
    "Text:\n{text}\n\n"
    "Create exactly {n} multiple-choice questions for '{subject}' at '{grade}' level.\n"
    "Tone: {tone}. Ensure factual correctness from the given text only.\n"
    "Return JSON using the following example as the desired shape:\n"
    "{example}\n"
)

EXAMPLE_JSON = json.dumps({
    "1": {"mcq": "...", "options": {"a": "...", "b": "...", "c": "...", "d": "..."}, "correct": "a", "explanation": "..."}
}, indent=2)


In [35]:
SYSTEM_REVIEW = {
    "role": "system",
    "content": (
        "You are an expert English grammarian and pedagogue. Output ONLY valid JSON.\n"
        "Return an object with two keys:\n"
        "1) complexity_analysis: a single short string (max 50 words) describing the quiz complexity.\n"
        "2) updated_quiz: a JSON object in the same shape as the original quiz (questions/options/correct/explanation). "
        "If a question needs rewriting to match the target students, update it; otherwise keep it identical.\n"
        "Do not add extra keys or extraneous text."
    )
}
REVIEW_USER_TEMPLATE = (
    "Subject: {subject}\n"
    "Grade level / audience description: {grade}\n"
    "Tone: {tone}\n\n"
    "Here is the quiz JSON to evaluate and (if necessary) modify:\n\n"
    "{quiz_json}\n\n"
    "Requirements:\n"
    "- Give a single short complexity_analysis (max 50 words).\n"
    "- If any question is inappropriate for the stated grade/ability, provide an updated version inside updated_quiz.\n"
    "- Return only the required JSON structure."
)

In [ ]:
file_path=r"-----"
with open(file_path, "r", encoding="utf-8") as f:
    TEXT = f.read()

In [37]:
NUMBER = 5
SUBJECT = "Social Science"
TONE = "simple"
GRADE = "high-school"

In [ ]:
user_msg_gen = BASE_USER_TEMPLATE.format(text=TEXT, n=NUMBER, subject=SUBJECT, tone=TONE, example=EXAMPLE_JSON,grade=GRADE)
messages = [
    SYSTEM_GEN,
    {"role": "user", "content": user_msg_gen}
]

try:
    print("Requesting quiz generation...")
    gen_content, gen_raw, gen_headers = groq_chat(messages, temperature=0.25, max_tokens=1500, timeout=120)
    if not gen_content:
        raise RuntimeError("No content returned from generation API call.")
    try:
        quiz_json = json.loads(gen_content)
    except json.JSONDecodeError:
        import re
        m = re.search(r"\{(?:.|\n)*\}\s*$", gen_content)
        if m:
            quiz_json = json.loads(m.group(0))
        else:
            raise
    print("Quiz generated successfully. Parsed JSON keys:", list(quiz_json.keys()))
except Exception as e:
    print("Error during quiz generation:", str(e))
    traceback.print_exc()
    raise


Requesting quiz generation...
Quiz generated successfully. Parsed JSON keys: ['1', '2', '3', '4', '5']


In [ ]:
review_user_content = REVIEW_USER_TEMPLATE.format(subject=SUBJECT, grade=GRADE, tone=TONE,
                                                 quiz_json=json.dumps(quiz_json, indent=2))
messages_review = [SYSTEM_REVIEW, {"role": "user", "content": review_user_content}]

try:
    print("\n== Requesting quiz review ==")
    review_content, review_raw, review_headers = groq_chat(messages_review, temperature=0.0, max_tokens=800, timeout=120)
    if not review_content:
        raise RuntimeError("No content returned from review API call.")
    try:
        review_json = json.loads(review_content)
    except json.JSONDecodeError:
        import re
        m = re.search(r"\{(?:.|\n)*\}", review_content)
        if m:
            review_json = json.loads(m.group(0))
        else:
            raise

    complexity = review_json.get("complexity_analysis")
    updated_quiz = review_json.get("updated_quiz")

    print("== Complexity analysis (<=50 words) ==")
    print(complexity or "No complexity analysis returned.")

    final_quiz = updated_quiz if updated_quiz else quiz_json

    print("\n== Final quiz JSON ==")
    print(json.dumps(final_quiz, indent=2, ensure_ascii=False))

    print("\n== Raw review JSON ==")
    print(json.dumps(review_json, indent=2, ensure_ascii=False))

    print(f"\nFinished: final quiz contains {len(final_quiz.keys())} questions.")
except Exception as e:
    print("Error during quiz review:", str(e))
    traceback.print_exc()
    raise


== Requesting quiz review ==
== Complexity analysis (<=50 words) ==
This quiz assesses high school students' knowledge of social science topics, including ancient civilizations, empires, and modern India's history and wildlife conservation.

== Final quiz JSON ==
{
  "1": {
    "mcq": "When did humans first arrive in India?",
    "options": {
      "a": "55,000 years ago",
      "b": "9,000 years ago",
      "c": "1200 BCE",
      "d": "400 BCE"
    },
    "correct": "a",
    "explanation": "According to the text, humans arrived in India from Africa no later than 55,000 years ago."
  },
  "2": {
    "mcq": "What civilization emerged in the Indus river basin?",
    "options": {
      "a": "Maurya Empire",
      "b": "Gupta Empire",
      "c": "Indus Valley Civilisation",
      "d": "Vijayanagara Empire"
    },
    "correct": "c",
    "explanation": "The text states that settled life emerged in the Indus river basin 9,000 years ago, evolving into the Indus Valley Civilisation."
  },
  "